In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from dask.distributed import Client
import dask
import xarray as xr
import xgcm
from dask.diagnostics import ProgressBar
import sys
import matplotlib.patches as patches
sys.path.append('/homes/metofac/wenegrat/xroms/')
sys.path.append('/homes/metofac/wenegrat/pyspec/')

#from xroms import open_roms_netcdf_dataset
import cmocean.cm as cmo
import xroms
from pyspec import spectrum as spec
from scipy import integrate as integrate
from timeit import default_timer as timer

#%%
plt.rcParams['text.usetex'] = True
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 14
plt.rcParams['contour.negative_linestyle'] = 'solid'

import sys
sys.path.append("../src/")
import load_model_runs as lruns

from importlib import reload  
import time

In [2]:
#client = Client(n_workers=2, threads_per_worker=1, memory_limit='40GB')
#client = Client(n_workers=6, threads_per_worker=1, memory_limit='10GB')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35626 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 64 Memory: 270.46 GB


In [3]:
reload(lruns)
dsOLD, gridOLD = lruns.loadRun('../data/model/jet_his_20_NC.nc', chunks={'s_rho':10, 's_w':10}, old=True)
dsOLD

<xarray.Dataset>
Dimensions:     (auxil: 4, eta_rho: 4002, eta_v: 4001, ocean_time: 41, s_rho: 60, s_w: 61, xi_rho: 1002, xi_u: 1001)
Coordinates:
  * xi_rho      (xi_rho) int64 0 1 2 3 4 5 6 7 ... 995 996 997 998 999 1000 1001
  * xi_u        (xi_u) int64 0 1 2 3 4 5 6 7 ... 994 995 996 997 998 999 1000
  * eta_rho     (eta_rho) int64 0 1 2 3 4 5 6 ... 3996 3997 3998 3999 4000 4001
  * eta_v       (eta_v) int64 0 1 2 3 4 5 6 ... 3995 3996 3997 3998 3999 4000
  * s_rho       (s_rho) float32 -0.9917 -0.975 -0.9583 ... -0.025 -0.008333
  * s_w         (s_w) float32 -1.0 -0.9833 -0.9667 ... -0.03333 -0.01667 0.0
    x_rho       (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    y_rho       (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
  * ocean_time  (ocean_time) float64 1.4e+07 1.408e+07 ... 1.737e+07 1.745e+07
    z_w         (ocean_time, s_w, eta_rho, xi_rho) float32 dask.array<chunksize=(41, 10, 4002, 1002), meta=np.ndarray>
    z_w_u       (ocean_time, s_w, eta_rho, xi_u) float32 dask.array<chunksize=(41, 10, 4002, 1001), meta=np.ndarray>
    z_w_v       (ocean_time, s_w, eta_v, xi_rho) float32 dask.array<chunksize=(41, 10, 4001, 1002), meta=np.ndarray>
    z_w_psi     (ocean_time, s_w, eta_v, xi_u) float32 dask.array<chunksize=(41, 10, 4001, 1001), meta=np.ndarray>
    z_rho       (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(41, 10, 4002, 1002), meta=np.ndarray>
    z_rho_u     (ocean_time, s_rho, eta_rho, xi_u) float32 dask.array<chunksize=(41, 10, 4002, 1001), meta=np.ndarray>
    z_rho_v     (ocean_time, s_rho, eta_v, xi_rho) float32 dask.array<chunksize=(41, 10, 4001, 1002), meta=np.ndarray>
    z_rho_psi   (ocean_time, s_rho, eta_v, xi_u) float32 dask.array<chunksize=(41, 10, 4001, 1001), meta=np.ndarray>
    z_rho0      (s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(10, 4002, 1002), meta=np.ndarray>
    z_rho_u0    (s_rho, eta_rho, xi_u) float32 dask.array<chunksize=(10, 4002, 1001), meta=np.ndarray>
    z_rho_v0    (s_rho, eta_v, xi_rho) float32 dask.array<chunksize=(10, 4001, 1002), meta=np.ndarray>
    z_rho_psi0  (s_rho, eta_v, xi_u) float32 dask.array<chunksize=(10, 4001, 1001), meta=np.ndarray>
    z_w0        (s_w, eta_rho, xi_rho) float32 dask.array<chunksize=(10, 4002, 1002), meta=np.ndarray>
    z_w_u0      (s_w, eta_rho, xi_u) float32 dask.array<chunksize=(10, 4002, 1001), meta=np.ndarray>
    z_w_v0      (s_w, eta_v, xi_rho) float32 dask.array<chunksize=(10, 4001, 1002), meta=np.ndarray>
    z_w_psi0    (s_w, eta_v, xi_u) float32 dask.array<chunksize=(10, 4001, 1001), meta=np.ndarray>
Dimensions without coordinates: auxil
Data variables:
    spherical   |S1 ...
    xl          float32 ...
    el          float32 ...
    Vtransform  float32 ...
    sc_r        (s_rho) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    sc_w        (s_w) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    Cs_r        (s_rho) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    Cs_w        (s_w) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    hc          float32 ...
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    pm          (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(4002, 1002), meta=np.ndarray>
    time_step   (ocean_time, auxil) int32 dask.array<chunksize=(41, 4), meta=np.ndarray>
    scrum_time  (ocean_time) float64 dask.array<chunksize=(41,), meta=np.ndarray>
    u           (ocean_time, s_rho, eta_rho, xi_u) float32 dask.array<chunksize=(41, 10, 4002, 1001), meta=np.ndarray>
    v           (ocean_time, s_rho, eta_v, xi_rho) float32 dask.array<chunksize=(41, 10, 4001, 1002), meta=np.ndarray>
    rho         (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunk

In [ ]:
#reload(lruns)
#chunks = {'time':1, 's_rho':5}
#dsWC, gridWC = lruns.loadZarrRun('../data/interim/dst.zarr')
#dsWC['f'] = dsOLD.f
#dsWC

In [4]:
reload(lruns)
dsWC, gridWC = lruns.loadRun('../data/model/JET_NML_CFB/', chunks={'time':1}, old=False)
dsWC['f'] = dsOLD.f
#dsNC, gridNC = lruns.loadRun('../data/model/JET_ML_NOCFB/', chunks={'time':5, 's_rho':3}, old=False)
#dsNC['f'] = dsOLD.f

## Calculate the PV and Save to Disk

In [5]:
hboundary = 'fill'
sboundary = 'extend'
folder = 'pv_20_NC_extend'
tic = time.perf_counter()
dst = dsWC
dst = dst.swap_dims({'ocean_time':'time'})
#dst = dst.rename(ocean_time='time')
grid = gridWC
buoyancy = xroms.buoyancy(dst.rho, rho0=dst.rho0)
#buoyancy = buoyancy.swap_dims({'ocean_time': 'time'})
PV = xroms.ertel(
                buoyancy.chunk({'time':1, 's_rho':5, 'eta_rho':1001}),
                dst.u.chunk({'time':1, 's_rho':5, 'eta_rho':1001}),
                dst.v.chunk({'time':1, 's_rho':5, 'eta_v':1001}),
                dst.f,
                grid,
                hcoord="rho",
                scoord="s_rho",
                hboundary=hboundary,
                hfill_value=None,
                sboundary=sboundary,
                sfill_value=None,
            )
for i in range(0, 47, 5):
    PVT = PV.isel(time=i).compute()
    PVDS = PVT.to_dataset()
    for variable in PVDS.variables.values():
            try:
                variable.attrs.pop('grid')
            except:
                print('No grid')
#PVDS.chunk({'time':1, 's_rho':100, 'eta_rho':4002, 'xi_rho':1002}).to_zarr('../data/interim/pv.zarr', mode='w')
    PVDS.to_netcdf(f'../data/interim/{folder}/pv_{i}.nc')
    toc = time.perf_counter()
    print((toc-tic)/60)

No grid
No grid
No grid
No grid
6.836673996949988
No grid
No grid
No grid
No grid
14.119108141166665
No grid
No grid
No grid
No grid
21.187162909916637
No grid
No grid
No grid
No grid
27.768189519083293
No grid
No grid
No grid
No grid
34.989411925066634
No grid
No grid
No grid
No grid
47.145757141399976
No grid
No grid
No grid
No grid
53.55975607684995
No grid
No grid
No grid
No grid
59.71419404483331


IndexError: Index is not smaller than dimension 45 >= 42

In [ ]:
dst

In [ ]:
savePV = False


pvname = 'pv_wc'
dst = dsWC
dst = dst.swap_dims({'ocean_time':'time'})
grid = gridWC
buoyancy = xroms.buoyancy(dst.rho, rho0=dst.rho0)
#buoyancy = buoyancy.swap_dims({'ocean_time': 'time'})
#PV = xroms.ertel(
#                buoyancy,
#                dst.u,
#                dst.v,
#                dst.f,
#                grid,
#                hcoord="rho",
#                scoord="s_rho",
#                hboundary="extend",
#                hfill_value=None,
#                sboundary="extend",
#                sfill_value=None,
#            )

hboundary = 'extend'
sboundary = 'extend'
vort = xroms.relative_vorticity(
            dst.u,
            dst.v,
            grid,
            hboundary = hboundary,
            sboundary = sboundary,
            sfill_value = None
)
vort_g = xroms.to_grid(
            vort,
            grid,
            hcoord='rho',
            scoord='s_rho',
            hboundary = hboundary,
            sboundary = sboundary,
            hfill_value = None,
            sfill_value = None)

bz = xroms.ddz(
            buoyancy,
            grid,
            hcoord='rho',
            scoord='s_rho',
            hboundary = hboundary,
            sboundary = sboundary,
            hfill_value= None,
            sfill_value= None)
PV = (vort_g + dst.f)*bz
PV.name = 'PV'

vort.name= 'vort'

ux = xroms.hgrad(
    dst.u,
    grid,
    which ='xi',
    hboundary=hboundary,
    hfill_value=None,
    sboundary = sboundary,
    sfill_value=None)
with ProgressBar():
    PV = PV.to_dataset()
    if savePV:
        for variable in PV.variables.values():
            try:
                variable.attrs.pop('grid')
            except:
                print('No grid')
        PV.to_netcdf('../data/interim/pv_wc2.nc')

In [ ]:
q = dst.u
hfill_value = None
sfill_value = None
sboundary = 'extrapolate'
dqdx = grid.interp(
                grid.derivative(q, "X", boundary=hboundary, fill_value=hfill_value),
                "Z",
                boundary=sboundary,
                fill_value=sfill_value,
            )
dqdx.chunk({'time':1, 's_w':101, 'xi_rho':501, 'eta_rho':500}).isel(time=slice(0,1)).compute()

In [ ]:
#ux
with ProgressBar():
    UX = vort_g.to_dataset().chunk({'time':2,  'eta_rho':2})
    for variable in UX.variables.values():
            try:
                variable.attrs.pop('grid')
            except:
                print('No grid')
    UX.to_zarr('../data/interim/pv_ux.zarr', mode='w')
    #UX.to_netcdf('../data/interim/pv_ux.nc', engine='netcdf4')

In [ ]:
vort_g.isel(time=slice(0,1)).chunk(
    {'time':1, 's_rho':1000, 'xi_rho':5000, 'eta_rho':5000}).compute()

In [ ]:
tic = time.perf_counter()
ux.isel(time=slice(0,1)).chunk(
    {'time':1, 's_w':1000, 'xi_rho':5000, 'eta_rho':5000}).compute()
toc = time.perf_counter()
print((toc-tic)/60)

In [ ]:
tic = time.perf_counter()
ux.isel(time=slice(0,1)).chunk(
    {'time':1, 's_w':10, 'xi_rho':5000, 'eta_rho':5000}).compute()
toc = time.perf_counter()
print((toc-tic)/60)

In [ ]:
tic = time.perf_counter()
pv1 = vort.isel(time=0).compute()
toc = time.perf_counter()
print((toc-tic)/60)

In [ ]:
pv1

In [ ]:
hboundary = 'fill'
sboundary = 'fill'
tic = time.perf_counter()

vort = xroms.relative_vorticity(
            dst.u.chunk({'time':1, 's_rho':10, 'eta_rho':1000}),
            dst.v.chunk({'time':1, 's_rho':10, 'eta_v':1000}),
            grid,
            hboundary = hboundary,
            sboundary = sboundary,
            sfill_value = None
)

vort_g = xroms.to_grid(
            vort,
            grid,
            hcoord='rho',
            scoord='s_rho',
            hboundary = hboundary,
            sboundary = sboundary,
            hfill_value = None,
            sfill_value = None)

pv1 = vort_g.isel(time=0).compute()
toc = time.perf_counter()
print((toc-tic)/60)

In [ ]:
tic = time.perf_counter()

PVDS = PVT.to_dataset()
for variable in PVDS.variables.values():
        try:
            variable.attrs.pop('grid')
        except:
            print('No grid')
#PVDS.chunk({'time':1, 's_rho':100, 'eta_rho':4002, 'xi_rho':1002}).to_zarr('../data/interim/pv.zarr', mode='w')
PVDS.to_netcdf('../data/interim/pv.nc')
toc = time.perf_counter()
print((toc-tic)/60)

In [ ]:
from glob import glob
files = sorted(glob('../data/model/JET_ML_CFB/'+'jet_his.*.nc'))
dsWC = xr.open_mfdataset(files, combine='nested', concat_dim='time', data_vars='minimal', chunks={'time':1})

#ds = xr.open_dataset('../data/model/jet_his.nc')

In [ ]:
PVDS

In [ ]:
#Worked with 2 workers, 32 gb each
UX = dsWC
for variable in UX.variables.values():
        try:
            variable.attrs.pop('grid')
        except:
            print('No grid')
UX.to_zarr('../data/interim/dst.zarr', mode='w')